In [0]:
# from google.colab import drive

# drive.mount('/content/gdrive', force_remount = True)

In [0]:
!pip install torchvision
import torch
import torch.nn as nn
import torch.utils as utils
import torch.nn.init as init
from torch.autograd import Variable
import torchvision.utils as v_utils
import torchvision.datasets as dset
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import cv2
from collections import OrderedDict

### Hyperparameter

In [0]:
epoch = 100
batch_size = 16
learning_rate = 0.0002
num_gpus = 1

###Data Load

In [0]:
transform = transforms.Compose([ 
         transforms.Scale(64),                       
         transforms.ToTensor(),
         transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))                         
])

train_dataset = dset.CIFAR10(root='./data/', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:220: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
0it [00:00, ?it/s]

170500096it [00:06, 27062324.61it/s]                               


Extracting ./data/cifar-10-python.tar.gz to ./data/


###Generator

In [0]:
ngf = 64
class Generator(nn.Module):
    def __init__(self):
        super(Generator,self).__init__()
        self.layer1 = nn.Sequential(
             nn.ConvTranspose2d(100,ngf*8,4,1,0,bias=False),
             nn.BatchNorm2d(ngf*8),
             nn.ReLU(True),
        )
        self.layer2 = nn.Sequential(OrderedDict([
                        ('conv1', nn.ConvTranspose2d(ngf*8,ngf*4,4,2,1,bias=False)),
                        ('bn1', nn.BatchNorm2d(ngf*4)),
                        ('relu1', nn.LeakyReLU()),
                        ('conv2', nn.ConvTranspose2d(ngf*4,ngf*2,4,2,1, bias=False)),
                        ('bn2', nn.BatchNorm2d(ngf*2)),    
                        ('relu2', nn.LeakyReLU()),
            ]))
        self.layer3 = nn.Sequential(OrderedDict([
                        ('conv3',nn.ConvTranspose2d(ngf*2,ngf,4,2,1, bias=False)),
                        ('bn3',nn.BatchNorm2d(ngf)),    
                        ('relu3',nn.LeakyReLU()),
                        ('conv4',nn.ConvTranspose2d(ngf,3,4,2,1,bias=False)),
                        ('relu4',nn.Tanh())
            ]))

    def forward(self,z):
        out = self.layer1(z)
        # out = out.view(batch_size//num_gpus,256,7,7)
        out = self.layer2(out)
        out = self.layer3(out)
        return out

### Discriminator

In [0]:
ndf = 64
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator,self).__init__()
        self.layer1 = nn.Sequential(OrderedDict([
                        ('conv1',nn.Conv2d(3,ndf,4,2,1,bias=False)),   # batch x 16 x 28 x 28
                        #('bn1',nn.BatchNorm2d(8)),    
                        ('relu1',nn.LeakyReLU(0.2, inplace=True)),
                        ('drop1', nn.Dropout(0.2)),
                        ('conv2',nn.Conv2d(ndf,ndf*2,4,2,1,bias=False)),  # batch x 32 x 28 x 28
                        ('bn2',nn.BatchNorm2d(ndf*2)),    
                        ('relu2',nn.LeakyReLU(0.2, inplace=True)),
                        ('drop2', nn.Dropout(0.2))
        ]))
        self.layer2 = nn.Sequential(OrderedDict([
                        ('conv3',nn.Conv2d(ndf*2,ndf*4,4,2,1, bias=False)),  # batch x 64 x 14 x 14
                        ('bn3',nn.BatchNorm2d(ndf*4)),
                        ('relu3',nn.LeakyReLU(0.2, inplace=True)),
                        ('drop3', nn.Dropout(0.2)),
                        # ('max2',nn.MaxPool2d(2,2)),
                        ('conv4',nn.Conv2d(ndf*4, ndf*8, 4, 2, 1, bias=False)),  # batch x 128 x 7 x 7
                        ('bn4',nn.BatchNorm2d(ndf*8)),
                        ('relu4',nn.LeakyReLU(0.2, inplace=True)),
                        ('drop4', nn.Dropout(0.2))
        ]))
        self.fc = nn.Sequential(
                        nn.Conv2d(ndf*8,1,4,1,0,bias=False),
                        nn.Sigmoid()
        )

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        # out = out.view(batch_size//num_gpus, -1)
        out = self.fc(out)

        return out.view(-1,1).squeeze(1)
        # return out

### Check Layers

In [0]:
generator = nn.DataParallel(Generator()).cuda()
discriminator = nn.DataParallel(Discriminator()).cuda()

# 파라미터 확인하기 using class.state_dict().keys

gen_params = generator.state_dict().keys()
dis_params = discriminator.state_dict().keys()

for i in dis_params:
  print(i)

module.layer1.conv1.weight
module.layer1.conv2.weight
module.layer1.bn2.weight
module.layer1.bn2.bias
module.layer1.bn2.running_mean
module.layer1.bn2.running_var
module.layer1.bn2.num_batches_tracked
module.layer2.conv3.weight
module.layer2.bn3.weight
module.layer2.bn3.bias
module.layer2.bn3.running_mean
module.layer2.bn3.running_var
module.layer2.bn3.num_batches_tracked
module.layer2.conv4.weight
module.layer2.bn4.weight
module.layer2.bn4.bias
module.layer2.bn4.running_mean
module.layer2.bn4.running_var
module.layer2.bn4.num_batches_tracked
module.fc.0.weight


### Loss Function & Optimizer

In [0]:
loss_func = nn.BCELoss()

gen_optim = torch.optim.Adam(generator.parameters(), lr= 5*learning_rate,betas=(0.5,0.999))
dis_optim = torch.optim.Adam(discriminator.parameters(), lr=learning_rate,betas=(0.5,0.999))

ones_label = Variable(torch.ones(batch_size,1)).cuda()
zeros_label = Variable(torch.zeros(batch_size,1)).cuda()

# def image_check(gen_fake):
#     img = gen_fake.data.numpy()
#     for i in range(10):
#         plt.imshow(cv2.cvtColor(img[i][0], cv2.COLOR_BGR2RGB))
#         plt.show()

def image_check(gen_fake):
    img = gen_fake.data.numpy()
    fig, axs = plt.subplots(3,3, figsize=(10,10))
    axs = axs.ravel()
    for i in range(9):
        t_img = np.moveaxis(img[i], 0, -1) *(-1)
        axs[i].imshow(cv2.cvtColor(t_img, cv2.COLOR_BGR2RGB))
    plt.show()


### Train Model

In [9]:
for i in range(epoch):
    gen_loss_list = []
    dis_loss_list = []
    for j,(image,label) in enumerate(train_loader):
        #discriminator
        dis_optim.zero_grad()
        image = Variable(image).cuda()
        dis_real = discriminator.forward(image).view(-1,1)
        dis_real_error = torch.sum(loss_func.forward(dis_real,ones_label)) #진짜를 진짜와 비교했을 때 에러값
        dis_real_error.backward()
        # dis_optim.step()
        gen_loss = 0
        #generator
        z = Variable(torch.Tensor(batch_size,100,1,1).normal_(0,1)).cuda() #noise
        gen_fake = generator.forward(z) #generator로 생성한 fake image
        dis_fake = discriminator.forward(gen_fake.detach()).view(-1,1) #fake를 discriminate
        dis_fake_error = torch.sum(loss_func.forward(dis_fake,zeros_label)) #fake를 fake라고 한 error
        dis_fake_error.backward(retain_graph=True) 

        dis_loss = dis_real_error + dis_fake_error #진짜를 진짜로, 가짜를 가짜로 한 거. loss가 적어야 discriminator가 제대로 작동하지 않는 거
        # if gen_loss.data <=10:
        dis_optim.step()


        #generator로 다시
        gen_optim.zero_grad()
        gen_dis_fake = discriminator.forward(gen_fake).view(-1,1) #가짜 이미지에 대한 판단.
        gen_loss = torch.sum(loss_func.forward(gen_dis_fake, ones_label)) #가짜를 진짜라고 한 에러. 이게 낮아야 generator가 제대로 작동한 거
        gen_loss.backward()
        if dis_loss.data <= 5:
          gen_optim.step()
    
        # model save
        if i!=0 and j % 100 == 0:
            gen_loss_list.append(gen_loss.data)
            dis_loss_list.append(dis_loss.data)
            # print(gen_loss,dis_loss)
            # torch.save([generator,discriminator],'./model/dcgan.pkl')

            print("{}th iteration gen_loss: {} dis_loss: {}".format(i,gen_loss.data,dis_loss.data))
            # v_utils.save_image(gen_fake.data[0:25],"/content/gdrive/data/result/gen_{}_{}.png".format(i,j), nrow=5)
            
    image_check(gen_fake.cpu())

Output hidden; open in https://colab.research.google.com to view.

In [0]:
# for i in range(epoch):
#     for j,(image,label) in enumerate(train_loader):
#         image = Variable(image).cuda()
        
#         # generator
#         gen_optim.zero_grad()
        
#         z = Variable(torch.Tensor(batch_size,100,1,1).normal_(0,1)).cuda()
#         gen_fake = generator.forward(z) #만들어진 fake image
#         dis_fake = discriminator.forward(gen_fake).view(-1,1)  #fake를 discriminator로 classify
        
#         gen_loss = torch.sum(loss_func(dis_fake,ones_label)) # 진짜라고 classified된 fake image(낮을 수록 학습 잘 됨)
#         gen_loss.backward(retain_graph=True)
#         gen_optim.step()
    
#         # discriminator
#         dis_optim.zero_grad()
        
#         z = Variable(torch.Tensor(batch_size,100,1,1).normal_(0,1)).cuda()
#         gen_fake = generator.forward(z)
#         dis_fake = discriminator.forward(gen_fake).view(-1,1)
#         dis_real = discriminator.forward(image).view(-1,1) #진짜 이미지를 진짜라고 classify          
#         dis_loss = torch.sum(loss_func(dis_fake,zeros_label)) + torch.sum(loss_func(dis_real,ones_label)) #가짜를 가짜라고, 진짜를 진짜라고 한 거. 높을수록 학습 잘 된거 
#         dis_loss.backward()
#         dis_optim.step()
    
#         # model save
#         if i!=0 and j % 100 == 0:
#             # print(gen_loss,dis_loss)
#             # torch.save([generator,discriminator],'./model/dcgan.pkl')

#             print("{}th iteration gen_loss: {} dis_loss: {}".format(i,gen_loss.data,dis_loss.data))
#             # v_utils.save_image(gen_fake.data[0:25],"/content/gdrive/data/result/gen_{}_{}.png".format(i,j), nrow=5)
            
#     image_check(gen_fake.cpu())